In [1]:
# Results Database 

In [2]:
%run "U1. Imports.ipynb"

In [3]:
db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [4]:
# Create entry dataframe
def create_entry_results(contestKey):
    # Read in contest standings
    contest_standings = pd.read_csv(os.path.join(baseball_path, "1. Results", "A. Contest Results", f"contest-standings-{contestKey}.csv"))
    
    # Entry results 
    entry_results = contest_standings[['Rank', 'EntryId', 'EntryName', 'TimeRemaining', 'Points', 'Lineup']].dropna()
    
    # Copy lineup column for manipulation
    entry_results['Lineup_Copy'] = entry_results['Lineup']
    
    # Prep for splitting positions
    for pos in ['1B ']:
        entry_results['Lineup_Copy'] = entry_results['Lineup_Copy'].str.replace(pos, "")
    for pos in ['2B ', '3B', 'C ', 'OF ', 'P ', 'SS ']:
        entry_results['Lineup_Copy'] = entry_results['Lineup_Copy'].str.replace(pos, "SPLIT ")  
    
    # Split columns
    entry_results[['1B', '2B', '3B', 'C', 'OF', 'OF.1', 'OF.2', 'P', 'P.1', 'SS']] = entry_results['Lineup_Copy'].str.split(' SPLIT ', expand=True)
        
    # Drop edited Lineup column
    entry_results.drop(columns={'Lineup_Copy'}, inplace=True)
    
    # Add contestKey 
    entry_results['contestKey'] = contestKey
    
    return entry_results

In [5]:
entry_results = create_entry_results(150868714)
entry_results.to_sql('entry_results', con=engine, index=False, if_exists='append')

1783

In [6]:
entry_results = create_entry_results(150868672) 
entry_results.to_sql('entry_results', con=engine, index=False, if_exists='append')

582

In [7]:
sql_query = 'SELECT * FROM entry_results WHERE "1B" = "Connor Joe"'
df = pd.read_sql_query(sql_query, con=engine)

In [19]:
sql_query = 'SELECT * FROM entry_results WHERE "Rank" = 1'
df = pd.read_sql_query(sql_query, con=engine)
df

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,1B,2B,3B,C,OF,OF.1,OF.2,P,P.1,SS,contestKey
0,1,3920652795,mikebass77,0,193.75,1B Connor Joe 2B Ozzie Albies 3B Jared Triolo C Carlos Perez OF Michael Harris II OF Jack Suwinski OF Miguel Andujar P Gerrit Cole P Tarik Skubal SS Marco Luciano,Connor Joe,Ozzie Albies,Jared Triolo,Carlos Perez,Michael Harris II,Jack Suwinski,Miguel Andujar,Gerrit Cole,Tarik Skubal,Marco Luciano,150868714
1,1,3920069045,biggbrd27 (3/6),0,145.55,1B Yandy Diaz 2B Brandon Lowe 3B Andruw Monasterio C Victor Caratini OF Josh Lowe OF Sal Frelick OF Manuel Margot P Zach Eflin P Wade Miley SS Taylor Walls,Yandy Diaz,Brandon Lowe,Andruw Monasterio,Victor Caratini,Josh Lowe,Sal Frelick,Manuel Margot,Zach Eflin,Wade Miley,Taylor Walls,150868672


In [9]:
# Create entry dataframe
def create_player_results(contestKey):
    # Read in contest standings
    contest_standings = pd.read_csv(os.path.join(baseball_path, "1. Results", "A. Contest Results", f"contest-standings-{contestKey}.csv"))
    
    # Player results 
    player_results = contest_standings[['Player', 'Roster Position', '%Drafted', 'FPTS']].dropna()    
        
    # Add contestKey 
    player_results['contestKey'] = contestKey
    
    return player_results

player_results = create_player_results(150868672)
player_results.head()

,Player,Roster Position,%Drafted,FPTS,contestKey
0,Zach Eflin,P,79.29%,23.05,150868672
1,Willy Adames,SS,45.62%,9.00,150868672
2,Tommy Edman,OF,42.59%,0.00,150868672
3,Wade Miley,P,41.58%,28.50,150868672
4,Griffin Canning,P,41.25%,17.05,150868672


In [10]:
player_results = create_player_results(150868714)
player_results.to_sql('player_results', con=engine, index=False, if_exists='append')

171

In [11]:
player_results = create_player_results(150868672)
player_results.to_sql('player_results', con=engine, index=False, if_exists='append')

75

In [12]:
sql_query = 'SELECT * FROM player_results WHERE "FPTS" > 15'
df = pd.read_sql_query(sql_query, con=engine)
df.head()

,Player,Roster Position,%Drafted,FPTS,contestKey
0,Tarik Skubal,P,57.88%,37.95,150868714
1,Ranger Suarez,P,22.43%,18.20,150868714
2,Michael Harris II,OF,22.10%,19.00,150868714
3,Grayson Rodriguez,P,18.34%,16.45,150868714
4,Gerrit Cole,P,13.57%,36.80,150868714


In [25]:
sql_query = """SELECT entry_results.*, player_results.FPTS, player_results.'%Drafted'
               FROM entry_results 
               LEFT JOIN player_results 
               ON entry_results."1B" = player_results.Player 
               AND entry_results.contestKey = player_results.contestKey 
               WHERE FPTS > 15"""

df = pd.read_sql_query(sql_query, con=engine)

In [14]:
def create_payouts(contestKey):
    # Read in payouts
    payouts = pd.read_csv(os.path.join(baseball_path, "2. Contests", "B. Payouts", f"Payouts {contestKey}.csv"))
    
    return payouts

In [15]:
payouts = create_payouts(150868714)
payouts.to_sql('payouts', con=engine, index=False, if_exists='append')

18

In [16]:
payouts = create_payouts(150868672)
payouts.to_sql('payouts', con=engine, index=False, if_exists='append')

16

In [ ]:
print(todaysdate)
sql_query = 'SELECT * FROM payouts WHERE "minPosition" = 1'
df = pd.read_sql_query(sql_query, con=engine)
df.head()

NameError: name 'todaysdate' is not defined

In [21]:

sql_query = """SELECT e.*, pr.payoutDescription
                FROM entry_results AS e
                LEFT JOIN payouts AS pr
                ON e.Rank BETWEEN pr.minPosition AND pr.maxPosition
                AND e.contestKey = pr.contestKey
                WHERE minPosition = 1"""

df = pd.read_sql_query(sql_query, con=engine)

df.head()

,Rank,EntryId,EntryName,TimeRemaining,Points,Lineup,1B,2B,3B,C,OF,OF.1,OF.2,P,P.1,SS,contestKey,payoutDescription
0,1,3920652795,mikebass77,0,193.75,1B Connor Joe 2B Ozzie Albies 3B Jared Triolo C Carlos Perez OF Michael Harris II OF Jack Suwinski OF Miguel Andujar P Gerrit Cole P Tarik Skubal SS Marco Luciano,Connor Joe,Ozzie Albies,Jared Triolo,Carlos Perez,Michael Harris II,Jack Suwinski,Miguel Andujar,Gerrit Cole,Tarik Skubal,Marco Luciano,150868714,$600.00
1,1,3920069045,biggbrd27 (3/6),0,145.55,1B Yandy Diaz 2B Brandon Lowe 3B Andruw Monasterio C Victor Caratini OF Josh Lowe OF Sal Frelick OF Manuel Margot P Zach Eflin P Wade Miley SS Taylor Walls,Yandy Diaz,Brandon Lowe,Andruw Monasterio,Victor Caratini,Josh Lowe,Sal Frelick,Manuel Margot,Zach Eflin,Wade Miley,Taylor Walls,150868672,$200.00


In [5]:
statsapi = pd.read_csv(os.path.join(baseball_path, "3. MLB API", "Stats API", "Stats API 2023.csv"), encoding='iso-8859-1')
statsapi.to_sql('statsapi', con=engine, index=False, if_exists='append')

239279

In [8]:

sql_query = """SELECT *
                FROM statsapi
                WHERE batter = 660670"""

df = pd.read_sql_query(sql_query, con=engine)

df.head()

,atBatIndex,inning,halfInning,outs,type,id,event,eventType,description,rbi,awayScore,homeScore,batter,batterName,batSide,pitcher,pitcherName,pitchHand,postOnFirst,postOnSecond,postOnThird,start,end,movementReason,gamePk,weather,wind,venue,date,away_name,home_name,game_date,game_type,venue_id
0,0,1,top,0,atBat,660670,Single,single,Ronald Acuna Jr. singles on a sharp line drive to right fielder Lane Thomas.,0,0,0,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",None,None,None,1B,None,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
1,12,2,top,1,atBat,518595,Fielders Choice,fielders_choice,"Ronald Acuna Jr. reaches on a fielder's choice, fielded by shortstop CJ Abrams. Travis d'Arnaud to 3rd. Orlando Arcia to 2nd. Fielding error by shortstop CJ Abrams.",0,1,0,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}","{'id': 606115, 'fullName': 'Orlando Arcia', 'link': '/api/v1/people/606115'}","{'id': 518595, 'fullName': ""Travis d'Arnaud"", 'link': '/api/v1/people/518595'}",2B,3B,r_adv_force,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
2,12,2,top,1,atBat,606115,Fielders Choice,fielders_choice,"Ronald Acuna Jr. reaches on a fielder's choice, fielded by shortstop CJ Abrams. Travis d'Arnaud to 3rd. Orlando Arcia to 2nd. Fielding error by shortstop CJ Abrams.",0,1,0,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}","{'id': 606115, 'fullName': 'Orlando Arcia', 'link': '/api/v1/people/606115'}","{'id': 518595, 'fullName': ""Travis d'Arnaud"", 'link': '/api/v1/people/518595'}",1B,2B,r_adv_play,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
3,12,2,top,1,atBat,660670,Fielders Choice,fielders_choice,"Ronald Acuna Jr. reaches on a fielder's choice, fielded by shortstop CJ Abrams. Travis d'Arnaud to 3rd. Orlando Arcia to 2nd. Fielding error by shortstop CJ Abrams.",0,1,0,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}","{'id': 606115, 'fullName': 'Orlando Arcia', 'link': '/api/v1/people/606115'}","{'id': 518595, 'fullName': ""Travis d'Arnaud"", 'link': '/api/v1/people/518595'}",None,1B,None,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
4,26,3,top,3,atBat,660670,Groundout,field_out,"Ronald Acuna Jr. grounds out, third baseman Jeimer Candelario to first baseman Dominic Smith.",0,3,1,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,None,None,None,None,None,None,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309


In [6]:
statsapi.head()

,atBatIndex,inning,halfInning,outs,type,id,event,eventType,description,rbi,awayScore,homeScore,batter,batterName,batSide,pitcher,pitcherName,pitchHand,postOnFirst,postOnSecond,postOnThird,start,end,movementReason,gamePk,weather,wind,venue,date,away_name,home_name,game_date,game_type,venue_id
0,0,1,top,0,atBat,660670,Single,single,Ronald Acuna Jr. singles on a sharp line drive to right fielder Lane Thomas.,0,0,0,660670,Ronald Acuna Jr.,R,571578,Patrick Corbin,L,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",NaN,NaN,NaN,1B,NaN,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
1,1,1,top,1,atBat,660670,Strikeout,strikeout,Matt Olson strikes out swinging.,0,0,0,621566,Matt Olson,L,571578,Patrick Corbin,L,NaN,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",NaN,1B,2B,r_stolen_base_2b,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
2,1,1,top,1,atBat,621566,Strikeout,strikeout,Matt Olson strikes out swinging.,0,0,0,621566,Matt Olson,L,571578,Patrick Corbin,L,NaN,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",NaN,NaN,NaN,NaN,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
3,2,1,top,1,atBat,663586,Walk,walk,Austin Riley walks.,0,0,0,663586,Austin Riley,R,571578,Patrick Corbin,L,"{'id': 663586, 'fullName': 'Austin Riley', 'link': '/api/v1/people/663586'}","{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",NaN,NaN,1B,NaN,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309
4,3,1,top,2,atBat,645277,Flyout,field_out,Ozzie Albies flies out sharply to right fielder Lane Thomas. Ronald Acuna Jr. to 3rd.,0,0,0,645277,Ozzie Albies,R,571578,Patrick Corbin,L,"{'id': 663586, 'fullName': 'Austin Riley', 'link': '/api/v1/people/663586'}",NaN,"{'id': 660670, 'fullName': 'Ronald Acuna Jr.', 'link': '/api/v1/people/660670'}",NaN,NaN,NaN,718780,"45 degrees, Sunny.","8 mph, Out To RF.",Nationals Park.,"March 30, 2023",Atlanta Braves,Washington Nationals,2023-03-30,R,3309


Database

Contests?
Salaries
Results
    Entries
    Players
Payouts
Leaderboards (not a priority)

Statcast (yearly)
Stats API (yearly)

Depth Charts
Rosters

FanGraphs

Weather

Projections 
    DFF
    Rotowire
